# Rotations

The purpose of this same notebook is to demonstrate the use of monochromatic pulses to create unitaries, when these are docomponsed into sequences of rotations between levels. We will use some functions in the pulses module that create the pulses that induce sequences of rotations. Then, we will use the solvers module to propagate the system and demonstrate how the pulses work (approximately). To clarify evertyhing, we quickly summarize here some theoretical results.

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import numpy as np
import matplotlib
import qutip as qt
import qutip_qip.operations as operations
from time import time as clocktime

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers

In [ ]:
qocttools.about()

In [ ]:
data = []

# Rotation operators

Any unitary evolution operator on a two-level system (TLS) can be written as a rotation operator in the form:
\begin{equation}
R_{\vec{n}}(\theta) = e^{-i\frac{\theta}{2}\vec{n}\vec{\sigma}}\,,
\end{equation}
times some global uninteresting phase factor.
$\vec{n}$ is a unit three-dimensional vector, and $\theta \in [0, 4\pi)$. For more than two levels, one can speak of rotation operators within two-dimensional subspaces, $R^{(ij)}_{\vec{n}}(\theta)$.

# A first approach to the problem: exact rotations using two monochromatic pulses

First, let us approach the problem assuming that we have a generic TLS that we can manipulate using two coupling operators, i.e. we have a Hamiltonian in the form:
\begin{equation}
H(t) = H_0 + V(t) = 
\left[\begin{array}{cc}
\varepsilon_0 & 0
\\
0 & \varepsilon_1
\end{array}
\right]
+  a\cos(\Omega t + \phi) \sigma_x + a\sin(\Omega t + \phi) \sigma_y\,.
\end{equation}
We work in resonance, meaning $\varepsilon_0-\varepsilon_1 = \Omega$. Note that this value can be negative. The two coupling operators are _oriented_ in the $\sigma_x$ and $\sigma_y$ _directions_ (this will be generalized later on). Also, note that the phase $\phi$ is the same for both perturbations, as it is their amplitude $a$. We can then rewrite the Hamiltonian given above as:
\begin{equation}
H(t) =
\left[\begin{array}{cc}
\varepsilon_0 & 0
\\
0 & \varepsilon_1
\end{array}
\right]
+ a
\left[\begin{array}{cc}
0 & e^{-i(\Omega t+\phi)}
\\
e^{i(\Omega t+\phi)} & 0
\end{array}
\right]
\end{equation}

Let us now solve the problem in the interaction picture, through the transformation:
\begin{equation}
T = e^{iH_0 t}
\end{equation}
This transformation has the virtue of leading to a time-indepenent Schrödinger equation:
\begin{equation}
\dot{\psi}(t) = -i a (\cos\phi\sigma_x + \sin\phi\sigma_y)\psi(t)
= -i a \vec{n}\vec{\sigma}\psi(t)\,,
\end{equation}
where $\vec{n} = (\cos\phi, \sin\phi, 0)$ is a unit vector. The solution is:
\begin{equation}
\psi(t) = e^{-iat\vec{n}\vec{\sigma}}\psi(0)\,.
\end{equation}
One can see how in this way we have implemented an exact rotation, whose amplitude is given by $\theta/2 = at$. Amplitude and operation time $a$ and $t$ are thus inversely related: one can implement very fast rotations using very large amplitudes; or put it differently, if one can only use small amplitudes, the operation time must be long.

Therefore, using an external field with the form given above, we can implement any rotation as long as its axis
does not contain any $z$ component. If it does, one can implement it making use of a concatenation of pulses, for example:
\begin{equation}
R_Z(\theta) = R_X(-\pi/2)R_Y(\theta)R_X(\pi/2)\,.
\end{equation}

However, let us now suppose that the perturbations that we can use are not $\sigma_x$ and $\sigma_y$, but some combinations of those:

\begin{equation}
H(t) = 
\left[\begin{array}{cc}
\varepsilon_0 & 0
\\
0 & \varepsilon_1
\end{array}
\right]
+ f_1(t) V_1 + f_2(t)V_2
\end{equation}
where:
\begin{align}
V_1 &= \left[
\begin{array}{cc}0 & \mu_1 \\ \mu_1^* & 0\end{array}
\right]\,,
\\
V_2 &= \left[
\begin{array}{cc}0 & \mu_2 \\ \mu_2^* & 0\end{array}
\right]\,.
\end{align}

Note that this case includes the previous one if we set $\mu_1 = 1$, $\mu_2 = -i$, and $f_1(t) = a\cos(\Omega t+\phi), f_2(t) = a\sin(\Omega t + \phi)$.

One can then prove that, given once again a generic rotation in the form:
\begin{equation}
R_{\vec{n}}(\theta) = e^{-i\frac{\theta}{2}\vec{n}\vec{\sigma}}\,,
\end{equation}
for $\vec{n}=(\cos(\phi), \sin(\phi))$, one can implement it with the previous Hamiltonian, by setting:

\begin{equation}
\left[\begin{array}{c}f_1(t)\\f_2(t)\end{array}\right] = 
\boldsymbol{\mu}^{-1}a
\left[\begin{array}{c}\cos(\Omega t+\phi)\\\sin(\Omega t+\phi)\end{array}\right]
\end{equation}

where the $\boldsymbol{\mu}$ matrix is:
\begin{equation}
\boldsymbol{\mu} = \left[\begin{array}{cc} {\rm Re}\mu_1 & {\rm Re}\mu_2 \\ -{\rm Im}\mu_1 & - {\rm Im}{\mu_2} \end{array}\right]
\end{equation}
and the total propagation time $T$ must be related to $a$ by:
\begin{equation}
a T = \frac{\theta}{2}
\end{equation}

The problem can be generalized to $N$-dimensional quantum systems and rotations within 2-dimensional subspaces. However, the induced rotations by those pulses will not be exact, due to _leakage_: the levels outside the 2-dimensional subspace that we are addressing interfere, and get populated, too. This leakage effect can be reduced if the amplitude is reduced (in fact, if the frequencies of the unwanted transitions are different from the one of the subspace, one can probably prove that the solution becomes exact in the limit of zero amplitude).

Finally, it should be stressed that, in this way, we implement a rotation _in the interaction representation_.

# A second approach to the problem: approximate rotations using one monochromatic pulses and the rotating wave approximation

The previous scheme permits to obtain exact rotations for any required duration $T$, even short ones (the shorter the duration, the higher the required amplitude). However, it requires using two independent perturbation operators $V_1$ and $V_2$. What happens if can only apply one? For that case, there is an alternative albeit approximate approach to get these rotations, using the so-called rotating wave approximation (RWA). 

We start by assuming a Hamiltonian in the form:
\begin{equation}
H(t) = 
\left[
\begin{array}{cc}\varepsilon_0 & 0 \\ 0 & \varepsilon_1 \end{array}
\right]
+ a\cos(\omega t + \phi) V =
\left[
\begin{array}{cc}\varepsilon_0 & 0 \\ 0 & \varepsilon_1 \end{array}
\right]
+ a \cos(\omega t + \phi)
\left[\begin{array}{cc}
0 & \mu_0
\\
\mu_0^* & 0
\end{array}
\right]\,
\end{equation}
where, as before, $\varepsilon_0-\varepsilon_1=\Omega$. Let us not make for the moment the resonance condition, and allow for $\omega \ne \Omega$.

In the interaction picture, the Hamiltonian reads:
\begin{equation}
H_I(t) = a\cos(\omega t + \phi)
\left[\begin{array}{cc}
0 & \mu_0 e^{i\Omega t}
\\
\mu_0^* e^{-i\Omega t} & 0
\end{array}
\right]
\end{equation}

The RWA consists in expanding $\cos(\omega t + \phi)$ as $\frac{1}{2}(e^{i(\omega t + \phi)} + e^{-i(\omega t t+ \phi)})$, and neglecting all terms containing $e^{i(\omega + \Omega)t}$. The resulting Hamiltonian is:
\begin{equation}
H_I(t) = \frac{1}{2}a\left[\begin{array}{cc}
0 & \vert\mu_0\vert e^{-i(\alpha + \delta t)}
\\
\vert\mu_0\vert e^{i(\alpha + \delta t)} & 0
\end{array}\right]
\end{equation}
where
\begin{align}
\alpha &= \arg \mu_0 + \phi
\\
\alpha &= \phi - \arg \mu_0
\\
\delta &= \omega - \Omega
\end{align}

Note that $\arg \mu_0$ is the parameter that _tunes_ the perturbation between $\sigma_x$ and $\sigma_y$:
\begin{equation}
V = \vert \mu_0 \vert \cos\arg\mu_0 \sigma_x - \vert \mu_0 \vert \sin\arg\mu_0\sigma_y\,.
\end{equation}
And note that the Hamiltonian $H_I$ only depends on the sum $\alpha = \arg\mu_0 + \phi$, and therefore it is not too important whether $V$ has $\sigma_x$ or $\sigma_y$ character, as one can always change $\phi$ accordingly.

Let us know make a new unitary transformation, using:
\begin{equation}
T_d(t) = \left[\begin{array}{cc}
e^{-i\frac{\delta}{2}t} & 0 
\\
0 & e^{i\frac{\delta}{2}t}
\end{array}\right]
\end{equation}
that leads to the _final_ Hamiltonian:
\begin{equation}
H'_I = \vec{m}\vec{\sigma}
\end{equation}
where
\begin{align}
m_1 &= \frac{1}{2}a\vert\mu_0\vert\cos\alpha\,,
\\
m_2 &= \frac{1}{2}a\vert\mu_0\vert\sin\alpha\,,
\\
m_3 &= \frac{1}{2}\delta\,.
\end{align}

Given that this Hamiltonian is time independent, the solution is trivial:
\begin{equation}
\psi(t) = \exp{(-i\vec{m}\vec{\sigma}t)}\psi(0)\,.
\end{equation}

We have, once again, a rotation operator. However, this is not a rotation operator in the interaction representation, which is the one that normally one is after, since we had to do the previous transformation that depends on the _dephasing_ $\delta$. If the detuning is zero (i.e., once again, we assume the resonance condition), we are back to a rotation $R_{\vec{n}}(\theta)$ in the interaction representation, where:
\begin{align}
\theta &= a \vert \mu_0 \vert t
\\
n_1 &= \cos\alpha
\\
n_2 &= \sin\alpha
\\
n_3 &= 0
\end{align}

Therefore, if we wish to implement a $R_{\vec{n}}(\theta)$ rotation _in the RWA_, we must (1) set the phase $\phi$ in such a way as to make $\cos\alpha, \sin\alpha = n_1, n_2$ (it is assumed that $n_3 = 0$), and set set the total operation time $T$ and the amplitude $a$ in such a way that they fulfill:
\begin{equation}
\theta = a \vert \mu_0 \vert T\,.
\end{equation}
A typical case is the need to implement the $U_X = \sigma_x$ gate, that flips the states of a qubit:
\begin{equation}
U_X = \left[\begin{array}{cc}0 & 1 \\ 1 & 0 \end{array}\right]
\end{equation}

This unitary is however _not_ a rotation, since its determinant is not one; it is however physically equivalent to:
\begin{equation}
-i U_X = R_{(1, 0, 0)}(\pi)
\end{equation}
In order to implement this gate, one must set $\alpha = \phi + \arg\mu_0 = 0$ (so that $\cos\alpha = 1$, and set $a$ and $T$ so that $\pi = \vert \mu_0\vert aT$ (this is called a $\pi$-pulse).

# Rotation sequences

One can prove that, except for a global phase factor, any unitary in a $N$-level system can be decomposed into a sequence of rotations:
\begin{equation}
U = R_{\vec{n}_K}(\theta_K)R_{\vec{n}_{K-1}}(\theta_{K-1})\dots R_{\vec{n}_1}(\theta_1)\,.
\end{equation}

For example, in an 8-level system, the Toffoli gate is given by:
\begin{align}
    U_{\rm Toffoli} = e^{i\frac{\pi}{8}} &
    R_{Z}^{(01)}(\frac{1}{4}\pi)
    R_{Z}^{(12)}(\frac{1}{2}\pi)
    R_{Z}^{(23)}(\frac{3}{4}\pi)
    R_{Z}^{(34)}(\pi)
    \\
    &
    R_{Z}^{(45)}(\frac{5}{4}\pi)
    R_{Z}^{(56)}(\frac{3}{2}\pi)
    R_{Z}^{(67)}(\frac{3}{4}\pi)
    R_{X}^{(67)}(\pi)
\end{align}

# Model

The system that we consider here is the GdW$_{30}$ complex [M. D. Jenkins et al., Physical Review B 95, 1 (2017)]. Its core is a Gd$^{3+}$ ion with a $4f^7$ configuration, whose ground manifold has $L = 0$ and $S = 7/2$. This $d$-level manifold ($d = 2S + 1 = 8$) can be considered a qudit. Under the effect of a DC magnetic field $\vec{H}$, the spin Hamiltonian of this molecule can be well approximated by an orthorhombic zero field splitting plus a Zeeman contribution:
\begin{equation}
H_0 = D\bigg[S_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[S_x^2 - S_y^2] - g\mu_B\vec{S}\cdot\vec{H}
\end{equation}
In our case, $S = 7/2$, $D$ = 1281 MHz, $E$ = 294 MHz, $\vec{H} = (0.15, 0.0, 0.0)$ T.

To this equation, we can add a time-dependent perturbation:
\begin{equation}
H(t) = H_0 + f_1(t)V_1 + f_2(t)V_2\,.
\end{equation}
The perturbations are magnetic field:
\begin{equation}
V_i = -g\mu_B\vec{S}\cdot\vec{H}_i \,.
\end{equation}

In [ ]:
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim
n = dim-1 # max level of the final state
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m1 = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H_m2 = np.array([0, 0, 0.001], dtype = float)
H0 = GdW30.hGdW30(D, E, H)
V1 = GdW30.vGdW30(H_m1)
V2 = GdW30.vGdW30(H_m2)
eigenvalues, eigenstates = H0.eigenstates()
H0 = H0.transform(eigenstates)
V1 = V1.transform(eigenstates)
V2 = V2.transform(eigenstates)
# The transformation may have "disordered" the levels. We have a diagonal matrix, and therefore
# the eigenvalues are the diagonal elements. The eigenvectors are the unit vectors in each direction.
eigenvalues = H0.diag()

Note that we are using, as perturbation operators, $\vec{H}_1$ in the $y$ direction, and $\vec{H}_2$ in the $z$ direction. In this way they are both linear combinations of $\sigma_x$ and $\sigma_y$ (within each 2-dimensional subspace). Now we compute all the transition frequencies (between nearest neighbours in energy). Since the most relevant
frequency for a Toffoli gate is the one between the sixth and the seventh levels, we will use that as the "reference" energy, $\Omega = \varepsilon_6 - \varepsilon_7$, and the reference period $\tau = \frac{2\pi}{\Omega}$.

In [ ]:
w = np.zeros(dim-1)
tau = np.zeros(dim-1)
for i in range(dim - 1):
    w[i] = eigenvalues[i] - eigenvalues[i+1] #in MHz
    tau[i] = 2.0*np.pi/w[i]

Omega = w[dim-2]
Tau = 2.0 * np.pi / np.abs(Omega)

Now we define the sequence of rotations. Each rotation is a list ``[axis, i, j, \theta]``, where ``axis`` is 0, 1, 2 depending on whether we want a $X$, $Y$, or $Z$ rotation, $i$ and $j$ are the levels addressed by the rotation, and $\theta$ is the angle. A sequence of rotations is obviously a list of rotations. We will use, for this example, the Toffoli sequence.

In [ ]:
xaxis = 0
yaxis = 1
zaxis = 2

toffoli_sequence = [ [xaxis, 6, 7, np.pi],
                     [zaxis, 6, 7, (3/4)*np.pi],
                     [zaxis, 5, 6, (3/2)*np.pi],
                     [zaxis, 4, 5, (5/4)*np.pi],
                     [zaxis, 3, 4, (1)*np.pi],
                     [zaxis, 2, 3, (3/4)*np.pi],
                     [zaxis, 1, 2, (1/2)*np.pi],
                     [zaxis, 0, 1, (1/4)*np.pi] ]

Instead of using the full Toffoli gate, we consider only the first two rotations (that, in fact, are four), to make things faster:

In [ ]:
target_sequence = toffoli_sequence[0:2]

Utarget = pulses.pulse_sequence_U(target_sequence, dim)
UToffoli = qt.Qobj(operations.toffoli().full())

# Time-propagations

We will now create the pulses objects that implement the sequence given above. We will fix the maximum amplitude; for the two-perturbation case, note how the two perturbations may have different amplitudes, and therefore we must speak of a maximum amplitude between the two. For the single perturbation case, the maximum amplitude is just the one amplitude of all the pulses of the sequence.

In [ ]:
amplitude = 1.0

In [ ]:
amp = 1.0 
f, maxcoeff = pulses.pulse_sequence2(target_sequence, amp, 0.0, V1, V2, eigenvalues)
f, maxcoeff = pulses.pulse_sequence2(target_sequence, (amplitude / maxcoeff) * amp, 0.0, V1, V2, eigenvalues)

T = f[0].T
ncycles = T / Tau
print("ncycles = {}".format(ncycles))

ntstepspercycle = 50
ntsteps = round(ntstepspercycle * ncycles)
ts = np.linspace(0, T, ntsteps)

t0 = clocktime()
result = solvers.solve('cfmagnus4', hamiltonians.hamiltonian(H0, [V1, V2]),
                       f, qt.identity(dim), ts,
                       interaction_picture = True,
                       returnQoutput = False)
t1 = clocktime()
print("Elapsed time = {}".format(t1-t0))

In [ ]:
fsinglecombined = pulses.pulse_sequence(target_sequence, amplitude, 0.0, V1, eigenvalues)

T = fsinglecombined.T
ncycles = T / Tau
print("ncycles (single) = {}".format(ncycles))

ntstepspercycle = 50
ntsteps = round(ntstepspercycle * ncycles)
ts = np.linspace(0, T, ntsteps)

t0 = clocktime()
result_single = solvers.solve('cfmagnus4', hamiltonians.hamiltonian(H0, [V1]),
                              fsinglecombined, qt.identity(dim), ts,
                              interaction_picture = True,
                              returnQoutput = False)
t1 = clocktime()
print("Elapsed time = {}".format(t1-t0))

As a measure of the fidelity of the induced unitary, we will use:
\begin{equation}
F(U) = \frac{1}{\rm dim}\vert {\rm Tr}\left[U^\dagger U_{\rm target}\right]\vert\,.
\end{equation}

In [ ]:
# Had we done the calculations in the Schrödinger representation, we would
# need to do a transformation here:
#res = (1j * H0 * ts[-1]).expm() * qt.Qobj(result[-1])

res = qt.Qobj(result[-1])
fu = np.abs(Utarget.overlap(res)/dim)
print("F(U) (double pulse) = {}".format(fu))
data.append(fu)

res = qt.Qobj(result_single[-1])
fu = np.abs(Utarget.overlap(res)/dim)
print("F(U) (single pulse) = {}".format(fu))
data.append(fu)

Note how neither of the two propagations have produced an exact result. In the two-perturbations case, the error is due to leakage; in the one-perturbation case, the error is due to both leakage and the rotating wave approximation.

In [ ]:
with open("data", "w") as f:
    for i in data:
        f.write("{:.14e}\n".format(i))